# Import libraries

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/ROBT613_BMI_Final_Project

/content/drive/MyDrive/BMI_Final_Project


In [3]:
!pip install mne==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313874 sha256=db91ce2ef09e7699ab66738d8e984924a6201bda7a8964fb3540716d80372a0a
  Stored in directory: /root/.cache/pip/wheels/50/55/a4/d95e3c6ab9370b77f9c871e9882a9578a2195c64ede88694ff
Successfully built mne


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import itertools
import pandas as pd
import pickle
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler

from nu_smrutils import loaddat,subject_specific,augment_dataset,crop_data,train_model

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

# Select device

In [5]:
#by default
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

# Select Dataset

## 1. BNCI2014004

In [6]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


# Load subject specific data, preprocess data (filtering, data augmentation, data cropping, standard scaling), and use Pytorch dataloaders

Function for standard scaler

In [7]:
scaler = StandardScaler()
def standard_scaling(data):
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  data = data.reshape(data.shape[0],-1)
  data = scaler.fit_transform(data)
  data = data.reshape(data.shape[0],shape1,shape2)
  return data

Function for Pytorch dataloaders

In [8]:
from torch.utils.data import TensorDataset, DataLoader

def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):
    # convert data dimensions to into to gray scale image format
    #if EEGNET: ### EEGNet model requires the last dimension to be 1
        #ff = lambda dat: torch.unsqueeze(dat, dim = -1)
    #else:
        #ff = lambda dat: torch.unsqueeze(dat, dim = 1)

    x_train, x_valid, x_test =  (dat['xtrain'],dat['xvalid'],dat['xtest'])
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)

    # TensorDataset & Dataloader
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev))
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev))

    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader},
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})
    return output

In [9]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Data cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
to

# LSTM architecture




In [10]:
class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim = 128, layer_dim = 1, output_dim = 2):
        super(LSTM_Model, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.fc(out[:, -1, :])

        return out

# Training loop

In [11]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = LSTM_Model(input_size[2])

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0109, acc: 0.4865
val loss: 0.0126, acc: 0.5145
Epoch 2/150
train loss: 0.0109, acc: 0.5163
val loss: 0.0126, acc: 0.5290
Epoch 3/150
train loss: 0.0108, acc: 0.5426
val loss: 0.0125, acc: 0.5652
Epoch 4/150
train loss: 0.0108, acc: 0.5590
val loss: 0.0125, acc: 0.5870
Epoch 5/150
train loss: 0.0107, acc: 0.5785
val loss: 0.0125, acc: 0.5870
Epoch 6/150
train loss: 0.0107, acc: 0.5939
val loss: 0.0125, acc: 0.5507
Epoch 7/150
train loss: 0.0106, acc: 0.6003
val loss: 0.0125, acc: 0.5471
Epoch 8/150
train loss: 0.0106, acc: 0.6106
val loss: 0.0125, acc: 0.5507
Epoch 9/150
train loss: 0.0105, acc: 0.6208
val loss: 0.0125, acc: 0.5435
Epoch 10/150
train loss: 0.0104, acc: 0.6279
val loss: 0.0125, acc: 0.5362
Epoch 11/150
train loss: 0.0104, acc: 0.6346
val loss: 0.0126, acc: 0.5507
Epoch 12/150
train loss: 0.0103, acc: 0.6436
val loss: 0.0126, acc: 0.5580
Epoch 13/150
train loss: 0.0102, acc: 0.6497
val loss: 0.0127,

# Save and print results

In [13]:
table.to_csv('results/LSTM/BNCI2014004_lstm.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.902564  0.641304      0.62    132
Subject2   0.775628  0.528736      0.38     43
Subject3   0.796474  0.550725      0.42     33
Subject4   0.886392  0.803509      0.88     38
Subject5   0.521848  0.564912      0.50      3
Subject6   0.933333  0.561594      0.56    110
Subject7   0.872436  0.612319      0.68    130
Subject8   0.903157  0.759450      0.74     47
Subject9   0.906410  0.583333      0.66    147

Average between subjects: 0.60
Max test_score between subjects: 0.88


Repeat the same steps for other datasets
## 2. BNCI2014001

In [14]:
itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


In [15]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.000001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Data cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

In [16]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-3
weight_decay = 1e-3
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = LSTM_Model(input_size[2])

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0110, acc: 0.5459
val loss: 0.0126, acc: 0.5766
Epoch 2/150
train loss: 0.0098, acc: 0.6868
val loss: 0.0135, acc: 0.5225
Epoch 3/150
train loss: 0.0081, acc: 0.7681
val loss: 0.0150, acc: 0.4955
Epoch 4/150
train loss: 0.0051, acc: 0.8889
val loss: 0.0208, acc: 0.4685
Epoch 5/150
train loss: 0.0027, acc: 0.9533
val loss: 0.0212, acc: 0.4955
Epoch 6/150
train loss: 0.0013, acc: 0.9831
val loss: 0.0302, acc: 0.3964
Epoch 7/150
train loss: 0.0007, acc: 0.9911
val loss: 0.0335, acc: 0.4595
Epoch 8/150
train loss: 0.0005, acc: 0.9952
val loss: 0.0352, acc: 0.4505
Epoch 9/150
train loss: 0.0004, acc: 0.9960
val loss: 0.0367, acc: 0.4324
Epoch 10/150
train loss: 0.0003, acc: 0.9968
val loss: 0.0381, acc: 0.4955
Epoch 11/150
train loss: 0.0003, acc: 0.9968
val loss: 0.0351, acc: 0.4775
Epoch 12/150
train loss: 0.0003, acc: 0.9976
val loss: 0.0400, acc: 0.4685
Epoch 13/150
train loss: 0.0002, acc: 0.9992
val loss: 0.0407,

In [17]:
table.to_csv('results/LSTM/BNCI2014001_lstm.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.545894  0.576577      0.46      1
Subject2   0.771337  0.639640      0.46     54
Subject3   0.774557  0.576577      0.48      3
Subject4   0.995169  0.522523      0.54     11
Subject5   0.971820  0.603604      0.48    138
Subject6   0.709340  0.549550      0.64      2
Subject7   0.896135  0.531532      0.50      4
Subject8   0.974235  0.540541      0.42      6
Subject9   0.996779  0.612613      0.54    111

Average between subjects: 0.50
Max test_score between subjects: 0.64


## 3. Weibo2014

In [18]:
itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


In [19]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Data cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
tor

In [20]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-3
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = LSTM_Model(input_size[2])

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0110, acc: 0.5217
val loss: 0.0115, acc: 0.4921
Epoch 2/150
train loss: 0.0100, acc: 0.6812
val loss: 0.0122, acc: 0.5238
Epoch 3/150
train loss: 0.0090, acc: 0.7319
val loss: 0.0138, acc: 0.5397
Epoch 4/150
train loss: 0.0081, acc: 0.7725
val loss: 0.0150, acc: 0.5556
Epoch 5/150
train loss: 0.0069, acc: 0.8333
val loss: 0.0156, acc: 0.5397
Epoch 6/150
train loss: 0.0054, acc: 0.8913
val loss: 0.0166, acc: 0.5714
Epoch 7/150
train loss: 0.0041, acc: 0.9348
val loss: 0.0203, acc: 0.5556
Epoch 8/150
train loss: 0.0026, acc: 0.9580
val loss: 0.0239, acc: 0.5238
Epoch 9/150
train loss: 0.0018, acc: 0.9768
val loss: 0.0285, acc: 0.5079
Epoch 10/150
train loss: 0.0015, acc: 0.9725
val loss: 0.0273, acc: 0.5556
Epoch 11/150
train loss: 0.0014, acc: 0.9797
val loss: 0.0339, acc: 0.4762
Epoch 12/150
train loss: 0.0007, acc: 0.9928
val loss: 0.0320, acc: 0.4762
Epoch 13/150
train loss: 0.0006, acc: 0.9928
val loss: 0.0372,

In [21]:
table.to_csv('results/LSTM/Weibo2014_lstm.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1    0.891304  0.571429      0.42      6
Subject2    0.963768  0.603175      0.46      9
Subject3    1.000000  0.619048      0.52     22
Subject4    0.926087  0.603175      0.46      7
Subject5    0.997101  0.619048      0.42     11
Subject6    0.978548  0.574074      0.54      9
Subject7    0.995652  0.634921      0.50     19
Subject8    0.972464  0.619048      0.62     32
Subject9    1.000000  0.730159      0.64     30
Subject10   1.000000  0.603175      0.64     16

Average between subjects: 0.52
Max test_score between subjects: 0.64
